<a href="https://colab.research.google.com/github/veiro/tesis-imputacion-datos/blob/main/codigo/utils/utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install "pandas<2.0.0

/bin/bash: -c: line 1: unexpected EOF while looking for matching `"'
/bin/bash: -c: line 2: syntax error: unexpected end of file


# imports

In [ ]:
import os
import sys

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from tqdm import tqdm
import pandas
from sklearn.model_selection import train_test_split
from datetime import datetime

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

import time
from tqdm import tqdm
from google.colab import drive
import pickle

# Version


In [ ]:
def version():
  return 30

# Mostrar datos


In [ ]:
def mostrarDatos(df_data, nombreDatos=''):
  '''
  print("DESCRIBE:\n")
  print(df_data.describe())
  print("\n--------------------------------------------------------------------------------\n")
  print("INFO:\n")
  print(df_data.info())
  print("\n--------------------------------------------------------------------------------\n")
 '''
  print("Nombre datos: "+ nombreDatos)
  print("\n--------------------------------------------------------------------------------\n")
  print(df_data.columns)

  print("\n--------------------------------------------------------------------------------\n")

  print("SHAPE:\n")
  print(df_data.shape)
  print("\n--------------------------------------------------------------------------------\n")

  print("Count of NaN:\n")
  print(df_data.isna().sum().sum())
  print("\n--------------------------------------------------------------------------------\n")

# Encode-decode


In [ ]:
# variables gloables
encoded_df_Country_Region_cols = 0
encoderOHE = OneHotEncoder(handle_unknown='ignore')
encoderLE1 = LabelEncoder()
encoderLE2 = LabelEncoder()
encoderLE3 = LabelEncoder()

def save_pickle_model(name, model, PATH_DATA_PROCESADA):
 with open(PATH_DATA_PROCESADA + '/' + name , 'wb') as handle:
    pickle.dump(model, handle, protocol=pickle.HIGHEST_PROTOCOL)

def load_pickle_model(name, PATH_DATA_PROCESADA):
  with open(PATH_DATA_PROCESADA + '/' + name, 'rb') as handle:
      model = pickle.load(handle)
  return model

def encoding(dataset, PATH_DATA_PROCESADA):
  global encoderOHE
  global encoderLE1
  global encoderLE2
  global encoderLE3
  global encoded_df_Country_Region_cols

  # ENCODING
  # 1- onehotenconding de Country_Region
  df=dataset

  encoded_data = encoderOHE.fit_transform( df[['Country_Region']])
  encoded_df_Country_Region = pandas.DataFrame(encoded_data, columns=encoderOHE.get_feature_names_out(['Country_Region']))
  encoded_df_Country_Region_cols = encoded_df_Country_Region.shape[1]

  df = df.drop(['Country_Region'], axis=1)

  # hice este concat horrible por que por alguna razon no me andaba bien el concant ordinario
  pos = encoded_df_Country_Region_cols
  for col in df.columns:
    encoded_df_Country_Region.insert( pos, col, df[col].to_numpy(), True)
    pos=pos+1

  df = encoded_df_Country_Region

  # 2 Province_State, Last_Update, date
  df['Province_State_encoded'] = encoderLE1.fit_transform(df['Province_State'])
  df = df.drop(['Province_State'], axis=1)

  df['Last_Update_encoded'] = encoderLE2.fit_transform(df['Last_Update'])
  df = df.drop(['Last_Update'], axis=1)

  df['date_encoded'] = encoderLE3.fit_transform(df['date'])
  df = df.drop(['date'], axis=1)

  df.reset_index(drop=True)

  # guardo los modelos utilizados:
  #save_pickle_model('encoderOHE',encoderOHE, PATH_DATA_PROCESADA )
  #save_pickle_model('encoderLE1',encoderLE1, PATH_DATA_PROCESADA)
  #save_pickle_model('encoderLE2',encoderLE1, PATH_DATA_PROCESADA)
  #save_pickle_model('encoderLE3',encoderLE1, PATH_DATA_PROCESADA)
  #save_pickle_model('encoded_df_Country_Region_cols',encoderLE1, PATH_DATA_PROCESADA)
  return df

# paso los datos imputados y los datos orignales transformados para hacer validaciones de formato y arreglar incosistencias por la imputacion.
def decode(imputed_data, data_encoded, PATH_DATA_PROCESADA):

  encoderOHE = load_pickle_model('encoderOHE', PATH_DATA_PROCESADA)
  encoderLE1 = load_pickle_model('encoderLE1', PATH_DATA_PROCESADA)
  encoderLE2 = load_pickle_model('encoderLE2', PATH_DATA_PROCESADA)
  encoderLE3 = load_pickle_model('encoderLE3', PATH_DATA_PROCESADA)
  encoded_df_Country_Region_cols = load_pickle_model('encoded_df_Country_Region_cols', PATH_DATA_PROCESADA)

  # esta linea la agregue para evitar problemas en data_encoded
  data_transformed = data_encoded.copy()
  df = pandas.DataFrame(imputed_data, columns = data_transformed.columns)
  # DECODE
  # 1- onehotenconding de Country_Region.
  encoded_df_Country_Region = df.iloc[:,  :encoded_df_Country_Region_cols]

  df['Country_Region'] = encoderOHE.inverse_transform(encoded_df_Country_Region).reshape(-1)
  df.drop(columns=df.columns[:encoded_df_Country_Region_cols], axis=1,  inplace=True)


  #2- Province_State, Last_Update, date

  Province_State_encoded_count = data_transformed.Province_State_encoded.unique().tolist().count(1)
  Last_Update_encoded_count = data_transformed.Last_Update_encoded.unique().tolist().count(1)
  date_encoded_encoded_count = data_transformed.date_encoded.unique().tolist().count(1)

  Province_State_encoded_most_freq = df['Province_State_encoded'].value_counts().idxmax()
  Last_Update_encoded_most_freq = df['Last_Update_encoded'].value_counts().idxmax()
  date_encoded_most_freq = df['date_encoded'].value_counts().idxmax()
  Country_Region_most_freq= df['Country_Region'].value_counts().idxmax()

  # esto es por si el metodo asigno una valor real, pero las codificaciones validacioenes son enterasm
  df['Province_State_encoded'] = df['Province_State_encoded'].astype(int)
  df['date_encoded'] = df['date_encoded'].astype(int)
  df['Last_Update_encoded'] = df['Last_Update_encoded'].astype(int)

  for i,  row in tqdm(df.iterrows()):
    # en algunos casos inventa valores que no existen, entonces como es un error le asigno el primero pero puede ser cualquier valor.

    if(row['Province_State_encoded'] < 1 or row['Province_State_encoded'] > Province_State_encoded_count ):
      #significa que genero algo que no existe, pongo el mas frequente
      df.at[i,'Province_State_encoded'] = Province_State_encoded_most_freq


    if(row['Last_Update_encoded'] <1 or row['Last_Update_encoded'] > Last_Update_encoded_count ):
      #significa que genero algo que no existe, pongo el mas frequente
      df.at[i,'Last_Update_encoded'] = Last_Update_encoded_most_freq


    if(row['date_encoded'] <1 or row['date_encoded'] > date_encoded_encoded_count):
      #significa que genero algo que no existe, pongo el mas frequente
      df.at[i,'date_encoded'] = date_encoded_most_freq

    if(row['Country_Region'] is None):
        # fallo al predecir el valor para alguna columna del onehotencoding, si no hago nada agrega nan
        # voy a poner el mas frecuente en este caso
        df.at[i,'Country_Region'] = Country_Region_most_freq


  df['Province_State'] = encoderLE1.inverse_transform(df['Province_State_encoded'].astype(int) )
  df['Last_Update'] = encoderLE2.inverse_transform(df['Last_Update_encoded'].astype(int) )
  df['date'] = encoderLE3.inverse_transform(df['date_encoded'].astype(int) )



  df = df.drop(['Province_State_encoded'], axis=1)
  df = df.drop(['Last_Update_encoded'], axis=1)
  df = df.drop(['date_encoded'], axis=1)

  #3- ordeno columnas

  col = df.pop('Province_State')
  df.insert(0, col.name, col)

  col = df.pop('Country_Region')
  df.insert(1,  col.name, col)

  col = df.pop('Last_Update')
  df.insert(2,  col.name, col)

  # cambio tipo de dato

  df['Confirmed'] = df['Confirmed'].astype(int)

  df['Deaths'] = df['Deaths'].astype(int)


  # cantidad de cifras significativas>

  df["Lat"] = round(df['Lat'],6)
  df["Long_"] = round(df['Long_'],4)

  return df

# Obtener datos

In [ ]:
def obtenerDatosTrain(MODO_DESARROLLO, PATH_DATA_PROCESADA):
  train_data_encoded_open = pandas.read_csv(PATH_DATA_PROCESADA + '/csse_covid_19_data_train_encoded=True_missing=False_' + 'Desarrollo=False' + '.csv')
  train_data_encoded_missing_open = pandas.read_csv(PATH_DATA_PROCESADA + '/csse_covid_19_data_train_encoded=True_missing=True_' + 'Desarrollo=False'  + '.csv')
  mask_train_open = np.load(PATH_DATA_PROCESADA + '/csse_covid_19_data_train_mask_'+ 'Desarrollo=False'+ '.npy')

  if(MODO_DESARROLLO):
    train_data_encoded_open = train_data_encoded_open[:10000]
    train_data_encoded_missing_open = train_data_encoded_missing_open[:10000]
    mask_train_open = mask_train_open[:10000]

  return train_data_encoded_open, train_data_encoded_missing_open, mask_train_open.astype(int)


In [ ]:
def obtenerDatosTest(MODO_DESARROLLO, PATH_DATA_PROCESADA):
  test_data_encoded_open = pandas.read_csv(PATH_DATA_PROCESADA + '/csse_covid_19_data_test_encoded=True_missing=False_' + 'Desarrollo=False' + '.csv').to_numpy()
  test_data_encoded_missing_open = pandas.read_csv(PATH_DATA_PROCESADA + '/csse_covid_19_data_test_encoded=True_missing=True_' + 'Desarrollo=False'  + '.csv').to_numpy()
  mask_test_open = np.load(PATH_DATA_PROCESADA + '/csse_covid_19_data_test_mask_'+ 'Desarrollo=False'+ '.npy')

  if(MODO_DESARROLLO):
    test_data_encoded_open = test_data_encoded_open[:10000]
    test_data_encoded_missing_open = test_data_encoded_missing_open[:10000]
    mask_test_open = mask_test_open[:10000]

  return test_data_encoded_open, test_data_encoded_missing_open, mask_test_open.astype(int)

# Embbedinggs

Se baso en https://arxiv.org/pdf/1604.06737.pdf
y yo en  https://colab.research.google.com/drive/13U4YRIdEu7SWS1ttiJPSrSayQHNbIaT_?usp=sharing#scrollTo=qSoepAKnfmvN

In [ ]:
# Data processing
import pandas as pd
import numpy as np


In [ ]:
import pickle
def getEmbbeding(X_test, PATH_DATA_PROCESADA):
  X_test_emb = X_test.copy()
  path_embbedings=PATH_DATA_PROCESADA + '/embbedings_weights_' + '.pkl'
  #with open(path_embbedings , 'rb') as f:
      #col_cat_emb_open = pickle.load(f)
  col_cat_emb_open = pd.read_pickle(path_embbedings)
  for col  in col_cat_emb_open:
    X_test_emb = pd.merge(X_test_emb, col.get("cat_emb"), left_on=col.get("columna"), right_on=col.get("columna"), how='inner')
    #borro columnas que por ahora no uso, si quiero hacer la operacion inversa las necesito.

  return X_test_emb

# Metricas

## RMSE

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(ori_data, imputed_data, data_m):
  return np.sqrt(np.mean((imputed_data * data_m - ori_data * data_m)**2))



## similaridad de coseno

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity


def calcular_similitud_cosine_promedio(df1, df2):
    similitudes = []
    for index in range(df1.shape[1]):
        tupla_df1 = df1.loc[index].values.reshape(1, -1)
        tupla_df2 = df2.loc[index].values.reshape(1, -1)
        similitud = cosine_similarity(tupla_df1, tupla_df2)
        similitudes.append(similitud[0][0])
    similitud_promedio = np.mean(similitudes)
    return similitud_promedio

# obtener df imputados


In [ ]:
def obtener_df_imputados(ori_data, imputed_data, data_m):
  # data_m = 1 si era null o 0 si no era null
  ori_data_np = ori_data.to_numpy()
  imputed_data_np  = imputed_data.to_numpy()

  imputed_data_np = imputed_data_np * data_m + ori_data_np * (1- data_m)

  df1 = pd.DataFrame(ori_data_np, columns = ori_data.columns)
  df2 = pd.DataFrame(imputed_data_np, columns = ori_data.columns)
  return df1, df2


# prettyPrint

In [ ]:
def prettyPrintName(strat):
  if('Transformer' in  strat):
    return 'Transformer'
  elif('XGBoost' in  strat):
    return 'XGBoost'
  elif('Knn' in  strat):
    return 'Knn'
  elif('GAIN' in  strat):
    return 'GAIN'
  elif('MIDA' in  strat):
    return 'MIDASpy'
  elif('constant' in  strat):
    return 'constant'
  elif('most_frequent' in  strat):
    return 'most_frequent'
  elif('mean' in  strat):
    return 'mean'
  elif('median' in  strat):
    return 'median'
  elif('IterativeImputer' in  strat):
    return 'IterativeImputer'
  else:
    return strat